In [1]:
import csv
import base64
import time
import fnmatch
from datetime import datetime, timedelta
from github import Github, StatsContributor
from sustainbeat import awesome2py
from urllib.parse import urlparse 
from os import getenv, path
from dotenv import load_dotenv

In [2]:
def countdown(t):
    while t:
        mins, secs = divmod(t, 60)
        timeformat = '{:02d}:{:02d}'.format(mins, secs)
        print(timeformat, end='\r')
        time.sleep(1)
        t -= 1
    print('\n\n\n\n\n')

In [3]:
load_dotenv()
g = Github(getenv("GITHUB"))

In [4]:
repo_dict = awesome2py.AwesomeList(".awesome.md")
print(repo_dict)

Photovoltaics and Solar Energy
 - pvlib-python - A set of documented functions for simulating the performance of photovoltaic energy systems. [https://github.com/pvlib/pvlib-python] - pvfactors - Open source view-factor model for diffuse shading and bifacial PV modeling. [https://github.com/SunPower/pvfactors] - gsee - Global Solar Energy Estimator. [https://github.com/renewables-ninja/gsee] - PVMismatch - An explicit Python PV system IV & PV curve trace calculator which can also calculate mismatch. [https://github.com/SunPower/PVMismatch] - rdtools - An open source library to support reproducible technical analysis of time series data from photovoltaic energy systems. [https://github.com/NREL/rdtools] - Machine-Learning-for-Solar-Energy-Prediction - Predict the power production of a solar panel farm from weather measurements using machine learning. [https://github.com/ColasGael/Machine-Learning-for-Solar-Energy-Prediction] - elpv-dataset - A dataset of functional and defective solar c

 - Appropedia - Shares knowledge to build rich, sustainable lives. [https://www.appropedia.org/Welcome_to_Appropedia] - Awesome Clean Tech - Clean technology is any process, product, or service that reduces negative environmental impacts through significant energy efficiency improvements, the sustainable use of resources, or environmental protection activities. [https://github.com/nglgzz/awesome-clean-tech] - Awesome Open Climate Science - Awesome Open Atmospheric, Ocean, and Climate Science. [https://github.com/pangeo-data/awesome-open-climate-science] - Awesome Earth - "What can I do about the climate crisis?" Here are 326 things you can do. [https://github.com/philsturgeon/awesome-earth] - Awesome Open Geoscience - Curated from repositories that make our lives as geoscientists, hackers and data wranglers easier or just more awesome. [https://github.com/softwareunderground/awesome-open-geoscience] - Datasets APIs and open source projects related to Climate Change - A curated list of 

In [ ]:
with open('./csv/GitHub.csv', 'w', newline='') as csv_github:
    fieldnames = ['project_name','oneliner','project_namespace','project_url', 'topics', 'rubric', 'last_update', 'stargazers_count','dominating_languages', 'organization', 'languages', 'homepage', 'created' , 'license','total_commits_last_year','last_issue_closed', 'issue_labels','good_first_issue','contributors', 'organization_avatar', 'funding' ]
    writer = csv.DictWriter(csv_github, fieldnames=fieldnames)
    writer.writeheader()
    for r in repo_dict.rubrics:        
        for e in r.entries: 
            print("------------------------")
            print("Processing %s" % e.name)
            print("URL: %s" % e.url)
            if urlparse(e.url).netloc != 'github.com':
                print(" %s is not a GitHub project" % e.name)
                print("Skip processing")
                print("------------------------")
                continue
            try:
                
                remaining,limit  = g.rate_limiting
                resettime = g.rate_limiting_resettime
                
                if(remaining<20):
                    print("------------------------")
                    print("Waiting for more available GitHub requests:")
                    current_time = datetime.now().timestamp() 
                    countdown((int(resettime)-int(current_time)+2))
                    
                print("GitHub Requests | Limit: {}, Remaining: {}".format(limit, remaining))
                # Gather project information from GitHub
                # https://pygithub.readthedocs.io/en/latest/github_objects/Repository.html
                repo_path = urlparse(e.url).path.strip("/")
                user, project_name  = path.split(repo_path)
                repo = g.get_repo(repo_path)
                files = repo.get_contents("")
                
                try: 
                    license = repo.get_license()
                    if license.license.spdx_id == 'NOASSERTION':
                        print("Custom license found")
                        license_name = "CUSTOM"
                    else:
                        license_name = license.license.spdx_id
                except:
                    print("No license information found")
                    license_name == "UNDEFINED"
                
                          
                #readme = base64.b64decode(repo.get_readme().content)
                issues = repo.get_issues(state='closed')

                ## preprocessing
                if issues.totalCount > 0:
                    last_issue_closed = issues[0].updated_at
                    
                labels = (",".join([entry.name for entry in repo.get_labels()])) 
                topics = (",".join(repo.get_topics())) 
                
                languages_states = repo.get_languages()
                programming_languages = (",".join(languages_states.keys()))
                dominating_languages = list(languages_states.keys())[0]
                
                try:
                    funding_file = repo.get_contents(".github")
                    for file in funding_file:
                        if file.path==".github/FUNDING.yml":
                            print("Funding found")
                            funding_information = base64.b64decode(file.content)
                        else:
                            funding_information = None
                except:
                    print("No funding information found")
                    funding_information = None


                if repo.organization is None: 
                    organization_name = None
                else: 
                    organization_name = repo.organization.login
                    organization_avatar = repo.organization.avatar_url

                total_commits_last_year = 0
                activity = repo.get_stats_commit_activity()
                for event in activity:
                    total_commits_last_year = total_commits_last_year + event.total
                    
                last_issue_delta = datetime.utcnow() - last_issue_closed
                
                contributors = repo.get_stats_contributors()
                contributor_activity = {}
                commits_total = 0
                for individuum in contributors:
                    contributor_activity[individuum.author.login] = individuum.total
                    commits_total = commits_total + individuum.total
            
                print(dict(sorted(contributor_activity.items(), key=lambda item: item[1])))
                
                # Do not process inactive projects
                if last_issue_delta > timedelta(days=365) and total_commits_per_year == 0:
                    print("%s is an inactive project" % e.name)
                    print("Skip processing")
                    print("------------------------")
                    continue
                
                entry_data = {    'project_name': e.name,
                                  'project_namespace': user,
                                  'project_url': repo.clone_url,
                                  'rubric':r.key,
                                  'oneliner':e.text[2:],
                                  'topics':topics,
                                  'organization':organization_name,
                                  'created':repo.created_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'last_update':repo.updated_at.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'total_commits_last_year':total_commits_last_year,
                                  'last_issue_closed':last_issue_closed.strftime("%Y/%m/%d, %H:%M:%S"),
                                  'stargazers_count':repo.stargazers_count,
                                  'dominating_languages':dominating_languages,
                                  'languages':programming_languages,
                                  'homepage':repo.homepage,
                                  'issue_labels': labels,
                                  'good_first_issue':repo.get_issues(state='open',labels=['good first issue']).totalCount,
                                  'license':license_name,
                                  'contributors':repo.get_contributors().totalCount,
                                  'organization_avatar': organization_avatar,
                                  'funding': funding_information,

                                    }
                print(entry_data)
                writer.writerow(entry_data)
                
            except Exception as e:
                print(e)
                
            


------------------------
Processing pvlib-python
URL: https://github.com/pvlib/pvlib-python
GitHub Requests | Limit: 5000, Remaining: 4054
{'aperetti': 1, 'markcampanelli': 1, 'cpr-chas': 1, 'YoannUniKS': 1, 'xcorail': 1, 'jetheurer': 1, 'alainakafkes': 1, 'dzimmanck': 1, 'mangecoeur': 1, 'hamiltonkibbe': 1, 'ncroft-b4': 1, 'EricDuminil': 1, 'KushalBeniwal': 1, 'AzizCode92': 1, 'btaute': 1, 'jkfm': 1, 'tsaoyu': 1, 'alamathe1': 1, 'hashstat': 1, 'roelloonen': 1, 'nappaillav': 1, 'dpostolovski': 1, 'modiprabal': 1, 'JohannesOos': 1, 'grisaitis': 1, 'rockstaedt': 1, 'birgits': 1, 'jurasofish': 1, 'odow': 2, 'emulbreh': 2, 'MichaelHopwood': 2, 'VolkrB': 2, 'Ahanmr': 2, 'tahentx': 2, 'kevinsa5': 2, 'pyElena21': 2, 'kahemker': 2, 'ericf900': 2, 'MLEEFS': 2, 'JPalakapillyKWH': 2, 'cedricleroy': 3, 'jranalli': 3, 'alexandermorgan': 3, 'jsstein': 4, 'tylunel': 4, 'Heliolytics': 4, 'jforbess': 4, 'anomam': 5, 'mayudong': 5, 'lboeman': 5, 'wfvining': 5, 'ejmiller2': 5, 'veronicaguo': 5, 'Konstant

{'project_name': 'feedinlib', 'project_namespace': 'oemof', 'project_url': 'https://github.com/oemof/feedinlib.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Contains implementations of photovoltaic models to calculate electricity generation from a PV installation based on given solar radiation. Furthermore it contains all necessary pre-calculations.', 'topics': '', 'organization': 'oemof', 'created': '2014/07/24, 09:03:29', 'last_update': '2021/08/21, 07:58:55', 'total_commits_last_year': 43, 'last_issue_closed': '2021/10/12, 15:25:58', 'stargazers_count': 64, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': '', 'issue_labels': 'bug,documentation,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 10, 'organization_avatar': 'https://avatars.githubusercontent.com/u/8503379?v=4', 'funding': None}
------------------------
Processing photovoltaic
URL: https://github.com/pvedu/photovoltaic

URL: https://github.com/PV-Lab/autoXRD
GitHub Requests | Limit: 5000, Remaining: 3864
No funding information found
{'dannyzekunren': 3, 'fipeop': 27}
{'project_name': 'autoXRD', 'project_namespace': 'PV-Lab', 'project_url': 'https://github.com/PV-Lab/autoXRD.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'A Python package for automatic XRD pattern classification of thin-films, tweaked for small and class-imbalanced datasets.', 'topics': '', 'organization': 'PV-Lab', 'created': '2019/04/23, 19:15:53', 'last_update': '2021/10/13, 11:11:59', 'total_commits_last_year': 0, 'last_issue_closed': '2021/10/04, 22:02:48', 'stargazers_count': 28, 'dominating_languages': 'Python', 'languages': 'Python', 'homepage': '', 'issue_labels': 'bug,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'Apache-2.0', 'contributors': 2, 'organization_avatar': 'https://avatars.githubusercontent.com/u/13911947?v=4', 'funding': None}
------

{'project_name': 'Charge Controller Firmware', 'project_namespace': 'LibreSolar', 'project_url': 'https://github.com/LibreSolar/charge-controller-firmware.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Firmware for LibreSolar MPPT/PWM charge controllers.', 'topics': '', 'organization': 'LibreSolar', 'created': '2016/08/02, 12:03:35', 'last_update': '2021/10/15, 02:19:14', 'total_commits_last_year': 72, 'last_issue_closed': '2021/06/08, 08:32:06', 'stargazers_count': 83, 'dominating_languages': 'C++', 'languages': 'C++,C,CMake,Batchfile,Shell,Python', 'homepage': 'https://libre.solar/charge-controller-firmware/', 'issue_labels': 'bug,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'Apache-2.0', 'contributors': 9, 'organization_avatar': 'https://avatars.githubusercontent.com/u/17674115?v=4', 'funding': None}
------------------------
Processing pvoutput
URL: https://github.com/openclimatefix/pvoutput
GitHub Requests | Limit: 5

{'project_name': 'rayflare', 'project_namespace': 'qpv-research-group', 'project_url': 'https://github.com/qpv-research-group/rayflare.git', 'rubric': 'Photovoltaics and Solar Energy', 'oneliner': 'Provide a flexible, user-friendly Python environment to model complex optical stacks, with a focus on solar cells.', 'topics': 'physics,optics,raytracing,ray-tracing,rigorous-coupled-wave,transfer-matrix-method,solar-cells,multiscale-simulation', 'organization': 'qpv-research-group', 'created': '2019/06/12, 20:01:41', 'last_update': '2021/10/11, 18:49:00', 'total_commits_last_year': 117, 'last_issue_closed': '2021/10/11, 18:48:55', 'stargazers_count': 4, 'dominating_languages': 'Python', 'languages': 'Python,TeX', 'homepage': '', 'issue_labels': 'bug,documentation,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'CUSTOM', 'contributors': 1, 'organization_avatar': 'https://avatars.githubusercontent.com/u/48552948?v=4', 'funding': 

...

{'project_name': 'Global Biotic Interactions', 'project_namespace': 'globalbioticinteractions', 'project_url': 'https://github.com/globalbioticinteractions/globalbioticinteractions.git', 'rubric': 'Life Forms and Biodiversity', 'oneliner': 'Global Biotic Interactions provides access to existing species interaction datasets.', 'topics': 'eol,ecology,biology,ecoinformatics,globi,species-interactions,bioinformatics,etl-framework,food-webs,pollinators,diseases,parasites,biodiversity,diet', 'organization': 'globalbioticinteractions', 'created': '2011/09/28, 21:51:47', 'last_update': '2021/10/14, 21:20:13', 'total_commits_last_year': 434, 'last_issue_closed': '2021/10/13, 21:11:55', 'stargazers_count': 76, 'dominating_languages': 'Java', 'languages': 'Java,HTML,Shell', 'homepage': 'https://globalbioticinteractions.org', 'issue_labels': 'bug,data management,dependencies,documentation,experimental,external issue,globi3,manual transcription,needs data review,needs parser,needs vocabulary / onto

Custom license found
No funding information found
{'lepittenger': 1, 'mmcfarland': 1, 'LarryLuTW': 1, 'cgarrard': 1, 'kdeloach': 1, 'ivanarellano': 1, 'mauriciopasquier': 1, 'riromu': 2, 'emstellato': 2, 'requires': 3, 'nicoali': 3, 'andrewbt': 3, 'bryant100000': 3, 'lederer': 6, 'hectcastro': 7, 'steventlamb': 24, 'balexandrowicz': 26, 'designmatty': 32, 'RobinIsTheBird': 110, 'jwalgran': 346, 'ahinz': 502, 'RickMohr': 600, 'maurizi': 885}
{'project_name': 'OpenTreeMap', 'project_namespace': 'OpenTreeMap', 'project_url': 'https://github.com/OpenTreeMap/otm-core.git', 'rubric': 'Life Forms and Biodiversity', 'oneliner': 'A collaborative platform for crowdsourced tree inventory, ecosystem services calculations, urban forestry analysis and community engagement.', 'topics': '', 'organization': 'OpenTreeMap', 'created': '2013/02/19, 15:11:04', 'last_update': '2021/10/16, 22:50:48', 'total_commits_last_year': 0, 'last_issue_closed': '2021/09/07, 23:56:01', 'stargazers_count': 163, 'dominati

{'project_name': 'forestmangr', 'project_namespace': 'sollano', 'project_url': 'https://github.com/sollano/forestmangr.git', 'rubric': 'Life Forms and Biodiversity', 'oneliner': 'An R package for forest mensuration and management.', 'topics': '', 'organization': None, 'created': '2018/10/08, 19:59:13', 'last_update': '2021/10/04, 00:47:56', 'total_commits_last_year': 33, 'last_issue_closed': '2021/08/15, 21:46:57', 'stargazers_count': 10, 'dominating_languages': 'R', 'languages': 'R', 'homepage': None, 'issue_labels': 'bug,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'CUSTOM', 'contributors': 1, 'organization_avatar': 'https://avatars.githubusercontent.com/u/5680763?v=4', 'funding': None}
------------------------
Processing CART
URL: https://github.com/jneme910/CART
GitHub Requests | Limit: 5000, Remaining: 187
No license information found
No funding information found
{'skyew': 1, 'SPeas': 2, 'jneme910': 387}
{'project_

{'project_name': 'forestatrisk', 'project_namespace': 'ghislainv', 'project_url': 'https://github.com/ghislainv/forestatrisk.git', 'rubric': 'Life Forms and Biodiversity', 'oneliner': 'A Python package to model and forecast the risk of deforestation.', 'topics': 'python,land-use-change,spatial-modelling,spatial-analysis,forecasting,spatial-autocorrelation,tropical-forests,roads,protected-areas,biodiversity-scenario,ipbes,co2-emissions,ipcc,forest-cover-change,deforestation,deforestation-risk,redd', 'organization': None, 'created': '2016/12/01, 10:45:12', 'last_update': '2021/10/02, 15:41:45', 'total_commits_last_year': 105, 'last_issue_closed': '2021/07/07, 10:03:12', 'stargazers_count': 36, 'dominating_languages': 'Python', 'languages': 'Python,C,TeX,Shell,Makefile,CSS', 'homepage': 'https://ecology.ghislainv.fr/forestatrisk', 'issue_labels': 'bug,duplicate,enhancement,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'GPL-3.0', 'contributors': 5, 'organization_

Processing CODED
URL: https://github.com/bullocke/coded
GitHub Requests | Limit: 5000, Remaining: 32
No funding information found
{'bullocke': 25}
{'project_name': 'CODED', 'project_namespace': 'bullocke', 'project_url': 'https://github.com/bullocke/coded.git', 'rubric': 'Life Forms and Biodiversity', 'oneliner': 'An algorithm developed to monitor for low-magnitude forest disturbances using Landsat data.', 'topics': '', 'organization': None, 'created': '2017/09/21, 14:59:30', 'last_update': '2021/09/20, 18:55:35', 'total_commits_last_year': 6, 'last_issue_closed': '2021/10/12, 22:50:56', 'stargazers_count': 27, 'dominating_languages': 'Python', 'languages': 'Python,Shell', 'homepage': '', 'issue_labels': 'bug,dependencies,duplicate,enhancement,good first issue,help wanted,invalid,question,wontfix', 'good_first_issue': 0, 'license': 'MIT', 'contributors': 1, 'organization_avatar': 'https://avatars.githubusercontent.com/u/38220006?v=4', 'funding': None}
------------------------
Processin